## What is it about

Taking small $n$ ($3\leqslant n \leqslant 8$), for each permutation $\pi \in S_n$ find a sequence $M$ of moves **L** (cyclic shift to the left), **R** (cyclic shift to the right), **X** (transposition of the first to elements) which converts $\pi$ to the identity permutation. Then if $M$ starts with **X**, find sequences which solve $L^{k}\pi$ for all $k = 1, \ldots, n-1$. The hypothesis is that the lengths of solving sequences for each pair $L^{k}\pi$, $L^{k+1}\pi$ are different. According to the calculations below, the hypothesis is only half correct — for even $n$.

In [1]:
!ls -l /kaggle/usr/lib/smallcg

total 392
-rw-r--r-- 1 nobody nogroup      0 Feb 16 10:51 custom.css
-rw-r--r-- 1 nobody nogroup    659 Feb 16 10:51 __output__.json
-rw-r--r-- 1 nobody nogroup 347399 Feb 16 10:51 __results__.html
-rw-r--r-- 1 nobody nogroup  15695 Feb 16 10:51 __script__.ipynb
-rw-r--r-- 1 nobody nogroup  14800 Feb 16 10:51 __script__.py
-rw-r--r-- 1 nobody nogroup  14800 Feb 16 10:51 smallcg.py


In [2]:
import numpy as np
from smallcg import TensorPerms

def get_LRX_moves(n):
    L = np.array( list(np.arange(1,n)) + [0])
    R = np.array( [n-1] + list(np.arange(n-1)) )
    X = np.array( [1,0] + list(np.arange(2,n)) )
    return np.array([L,R,X])

In [3]:
import torch

# n_generators = len( list_generators )
# state_size = len(list_generators[0] )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.uint8

In [4]:
from collections import deque

def beam_search_all(LRX, beam_width=1):
    result = dict()
    for i, state in enumerate(LRX.states):
        if i == 0:
            continue
        result[str(state.cpu().numpy())] = beam_search(LRX, state, beam_width)
    return result

def beam_search(LRX, state_start, beam_width=100, verbose=0):
    n = LRX.n
    n_steps_limit = 10*n**2
    # Main storage data:
    # Priority queue for the beam search; stores tuples of (cumulative cost, path, current state)
    queue_beam = deque([(0, [], state_start )] )

    # Set of all previously visited states 
    # We will ban visiting them again 
    set_seen_states = set( )
    set_seen_states.add(str(LRX.states[0]))

    counter_length = 0 # Path length counter 
    flag_path_found = False
    path_found = []

    while queue_beam: #  continue looping as long as queue is not empty. It should never be empty, unless graph is not conncected and path does not exists
        counter_length += 1
        #print('counter_length',counter_length)
    
        # Select top best states candidates - top-beam-width    
        queue_beam = deque(sorted(list(queue_beam), key=lambda x: x[0])[:beam_width])

        # Storage for newly generated states/costs/paths:
        # They will be obtained by applying all moves to all states in beam 
        queue_beam_next = deque()

        # That will be used just for monitoring - printing stat on obtained values 
        # Pay attention these are NOT cummulative costs 
        list_beam_q_values_for_monitoring = []

        # Loop via current beam:
        for cost, path, current_state in queue_beam:
            #print('cost, path, current_state',cost, path, current_state)

            # Loop via all possible moves 
            for i_action, action in enumerate(['L','R','X']):

                #Check conditions for reasonable moves, disregard unreasoanble
                # Condition proposed by S.Fironov - it is rather natural, is there theoretical proof ?
                # if action == 'X' and current_state[0] < current_state[1]: # Heuristics 
                #    continue            
                if action == 'L' and len(path) > 0 and path[-1] == 'R': # Non-backtracking
                    continue
                if action == 'R' and len(path) > 0 and path[-1] == 'L': # Non-backtracking
                    continue
 
                # Make a move (apply generator, i.e. go to neigbour node )
                generator =  LRX.moves[i_action, :]
                next_state = current_state[ generator ]

                # Check destnation found: 
                if torch.all( next_state == LRX.states[0]) :
                    flag_path_found = True
                    path_found = path + [action]
                    break 

                # Check the next_state is new or visited before 
                if str(next_state) not in set_seen_states:
                    set_seen_states.add(str(next_state))

                    # Compute heuristic distance to destination state
                    # Here it is Hamming distance, but can be neural net predictor:
                    # q_value = torch.sum( (next_state - state_destination ) !=0  ).item()
                    q_value = state2dist[str(next_state.cpu().numpy())]
                    list_beam_q_values_for_monitoring.append(q_value)

                    # Cummulative cost: 
                    total_cost = q_value # + alpha_previous_cost_accumulation * cost  
                    queue_beam_next.append((total_cost, path + [action], next_state))
                
                ###### End loop over actions/moves
        
            if flag_path_found: 
                break  # Path found. Stop process
                
        ###### End loop over beam
    
        if flag_path_found: break  # Path found. Stop process

        if verbose >= 100:
            if counter_length % 10 == 0:        
                print('Step:',counter_length, 'Visited states:', len(set_seen_states), 
                  'Beam min:',  np.min( list_beam_q_values_for_monitoring),
                  'median:',  np.median( list_beam_q_values_for_monitoring),
                  'max:',  np.max( list_beam_q_values_for_monitoring) )
            # print('new max len', len(path), len(queue), len(seen), 
            #       np.min(list_beam_q_values_for_monitoring), np.median(list_beam_q_values_for_monitoring),
            #       np.mean(list_beam_q_values_for_monitoring), np.max(list_beam_q_values_for_monitoring) )
        
    
        if counter_length > n_steps_limit:
            flag_path_found = False
            path_found = []
            break


        # Update the entire queu for new states/costs/paths:
        queue_beam = queue_beam_next

        ###### End loop iterations 
    return path_found

In [5]:
from time import time

for n in range(3, 9):
    begin = time()
    LRX = TensorPerms(n, get_LRX_moves(n), verbose=False)
    state2dist = dict()
    for i, state in enumerate(LRX.states):
        state2dist[str(state.cpu().numpy())] = int(LRX.distances[i].cpu().numpy())
    state2path = beam_search_all(LRX)
    equal_neighbors = 0
    for i, perm in enumerate(LRX.states):
        if i == 0:
            continue
        np_perm = str(perm.cpu().numpy())
        if len(state2path[np_perm]) != LRX.distances[i]:
            print(i, len(state2path[np_perm]), LRX.distances[i])
        if state2path[np_perm][0] == 'X':
            # print(i, np_perm)
            prev = 0
            for k in range(1, n):
                rolled = str(torch.roll(perm, k).cpu().numpy())
                rolled_path = state2path[rolled]
                if len(rolled_path) == prev: # and 2*k - 1 != n:
                    equal_neighbors += 1
                prev = len(rolled_path)
    print(n, equal_neighbors, f"took {time() - begin:.2f} s")

3 1 took 0.47 s
4 0 took 0.04 s
5 26 took 0.40 s
6 0 took 3.59 s
7 1522 took 36.77 s
8 0 took 416.87 s
